In [1]:
import pdal
import json

In [2]:
bounds = ([324000, 328500], [7357000, 7361000])

In [3]:
ept = [
    {
      "type": "readers.ept",
      "filename": "https://ept-m3dc-pmsp.s3-sa-east-1.amazonaws.com/ept.json",
      "resolution": 100.0,
      "bounds": str(bounds)
    }
]

pipeline = pdal.Pipeline(json.dumps(ept))
pipeline.validate()
%time n_points = pipeline.execute()
print(f'Pipeline selected {n_points} points')

CPU times: user 439 ms, sys: 32.1 ms, total: 471 ms
Wall time: 678 ms
Pipeline selected 5982 points


In [4]:
import pandas as pd
arr = pipeline.arrays[0]
df = pd.DataFrame(arr)
# print(df.head().to_latex(index=False))
df.head()

,Blue,Classification,EdgeOfFlightLine,GpsTime,Green,Intensity,NumberOfReturns,OriginId,PointSourceId,Red,ReturnNumber,ScanAngleRank,ScanDirectionFlag,UserData,X,Y,Z
0,19968,5,0,133583.836407,25344,57,1,1436,14,20480,1,13.0,1,12,327639.25,7360900.91,777.57
1,38912,19,0,133584.842278,38912,7,2,1435,14,39680,1,0.0,0,6,327493.97,7360900.24,774.44
2,61952,6,0,133585.298187,62464,45,1,1435,14,62208,1,-7.0,0,3,327426.76,7360901.21,775.04
3,36608,5,0,133586.852354,39936,46,1,1469,14,40448,1,6.0,1,3,327565.81,7360760.53,771.47
4,42752,6,0,133587.187101,40960,21,1,1469,14,40960,1,0.0,0,3,327496.91,7360759.29,773.05


In [5]:
# Reduzindo valores a mínimos para poderem ser visualizados
df[['X', 'Y', 'Z']] = df[['X', 'Y', 'Z']] - df[['X', 'Y', 'Z']].min()

In [6]:
import matplotlib.cm
import numpy as np

colormap = matplotlib.cm.viridis
c = colormap((df.Z-df.Z.mean())/df.Z.std())

In [7]:
c

array([[0.267004, 0.004874, 0.329415, 1.      ],
       [0.267004, 0.004874, 0.329415, 1.      ],
       [0.267004, 0.004874, 0.329415, 1.      ],
       ...,
       [0.993248, 0.906157, 0.143936, 1.      ],
       [0.525776, 0.833491, 0.288127, 1.      ],
       [0.668054, 0.861999, 0.196293, 1.      ]])

In [8]:
import ipyvolume as p3 

fig = p3.figure(width=1000)
fig.xlabel='Y'
fig.ylabel='Z'
fig.zlabel='X'
all_points = p3.scatter(df['Y'], df['Z'], df['X'], color=c, size=0.1, marker='square_2d')
# ground = p3.scatter(gd['Y'], gd['Z'], gd['X'], color='grey', size=.2)
# building = p3.scatter(bd['Y'], bd['Z'], bd['X'], color='orange', size=.3)
# vegetation = p3.scatter(vg['Y'], vg['Z'], vg['X'], color='green', size=.1)
p3.squarelim()
p3.show()